In [43]:
import random
import math
from math import sqrt

class StateVector:
    def __init__(self, location, velocity):
        self.time = 0 # Time of validity
        self.location = location # Generally a 3D vector
        self.velocity = velocity # Generally a 3D vector
    def update(self, newTime):
        dt = newTime - self.time
        self.time = newTime
        self.location[0] = self.location[0] + self.velocity[0] * dt/60.
        self.location[1] = self.location[1] + self.velocity[1] * dt/60.
        self.location[2] = self.location[2] + self.velocity[2] * dt/60.
    def __str__(self):
        return "StateVector at time " + str(self.time) + " is " + str(self.location) + " with velocity " + str(self.velocity)
    def __repr__(self):
        return self.__str__()
    

class LRDrone:
    def __init__(self, name, type):
        self.name = name
        self.state = StateVector([0,0,0], [0,0,0])
        self.detectedTargets = []
        self.remainingRange = 0 # km
        if type == "small":
            self.range = 300 #km
            self.sensor = ["EO/IR", "WBRFreceiver"]
            self.cruise_speed = 100 #km/h
            self.cruise_altitude = 3 #km
            self.communication = ["VHF", "UHF", "4G", "Wifi"]
        if type == "medium":
            self.range = 1000
            self.sensor = ["EO/IR", "WBRFreceiver"]
            self.cruise_speed = 200 #km/h
            self.cruise_altitude = 5 #km
            self.communication = ["VHF", "UHF", "4G", "Wifi", "Satellite"]
        if type == "fast":
            self.range = 200
            self.sensor = ["EO/IR", "WBRFreceiver", "HPRF transmitter"]
            self.cruise_speed = 600 #km/h
            self.cruise_altitude = 10 #km
            self.communication = ["VHF", "UHF", "4G", "Wifi", "Satellite"]
    
    
    def __str__(self):
        return "Drone " + self.name + " is type " + self.type + " with range " + str(self.range) + " km."
    
    def __repr__(self):
        return self.__str__()
    
class Target():
    def __init__(self, name, type):
        self.name = name
        self.type = type
        self.emissions = [] # list of emissions, electromagnetic (RF, thermal, visible), acoustic, etc.
        self.state = StateVector([0,0,0], [0,0,0])
    def __str__(self):
        return "Target " + self.name + " is type " + self.type + " at location " + str(self.location) + " with velocity " + str(self.velocity)
    def __repr__(self):
        return self.__str__()

# Scenario: 3 drones, 2 targets
# The location is a 250 km wide strait with shipping and aircraft traffic going up the middle
sensorRange = 200 #km

def distance(location1, location2):
    horizon1 = location1[2] * 6371. # km
    return sqrt((location1[0] - location2[0])**2 + (location1[1] - location2[1])**2 + (location1[2] - location2[2])**2)

earthRadius = 6371. # km
# Compute distance to the horizon from a given altitude in km
def distanceToHorizon(altitude):
    return sqrt(((altitude + earthRadius)**2)- (earthRadius**2)) # km

def isVisible(location1, location2):
    # Compute distance to horizon from location1
    d1 = distanceToHorizon(location1[2])
    d2 = distanceToHorizon(location2[2])
    d = distance(location1, location2)
    if d1 + d2 > d:
        return True
    else:
        return False

import logging
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='test_log.log', mode='a')
logger.addHandler(fhandler)
logging.warning('This is a warning message')

formatter = logging.Formatter('%(asctime)s - %(name)s - %(lineno)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.setLevel(logging.DEBUG)

# Create drones and targets
drone1 = LRDrone("drone1", "small")
droneLaunchSiteX = 0. 
droneLaunchSiteY = 0.
drone1.state = StateVector([droneLaunchSiteX, droneLaunchSiteY, drone1.cruise_altitude], [0,drone1.cruise_speed,0])

drones = []
drones.append(drone1)

target1 = Target("target1", "ship")
target1.state = StateVector([-300,125,0], [50,0,0])
target2 = Target("target2", "aircraft")
target2.state = StateVector([-300,125,0], [1000,0,0])
targets = []
targets.append(target1)
targets.append(target2)

# Simulation
detectedTargets = []
for t in range(0, 100):
    # Update drone locations
    for drone in drones:
        drone.state.update(t)
        # print(drone.state)
        # print("Drone " + drone.name + " location: " + str(drone.state.location) + " velocity: " + str(drone.state.velocity) + ".")
    for target in targets:
        # print("Target " + target.name + " location: " + str(target.state.location) + " velocity: " + str(target.state.velocity) + ".")
        target.state.update(t)

    for drone in drones:
        # Check distance to other drones
        # Check distance to targets
        for target in targets:
            if isVisible(drone.state.location, target.state.location):
                d = distance(drone.state.location, target.state.location)
                # print("Distance from drone " + drone.name + " to target " + target.name + " is " + str(d) + " km.")
                if d < sensorRange:
                    # Target is within sensor range
                    # Check for emissions
                    # Use a simple probability model for now
                    detectionProbability = 1.0
                    if random.random() < detectionProbability:
                        # Target is detected
                        # determine the target heading from the drone
                        targetHeading = math.atan2(target.state.location[1] - drone.state.location[1], target.state.location[0] - drone.state.location[0])
                        detectedTargets.append([t, d, drone, target, targetHeading])
                        print( str(t) + ", " + str(d) + ", " + str(int((180/math.pi)*targetHeading)) + ", " + 
                              target.name + ", " +
                              str(int(target.state.location[0])) + ", " + str(int(target.state.location[1])) + ", " + str(int(target.state.location[2])) + ", " + 
                              str(target.state.velocity[0]) + ", " + str(target.state.velocity[1]) + ", " + str(target.state.velocity[2]) + ", " +
                              drone.name + ", " +
                              str(drone.state.location[0]) + ", " + str(drone.state.location[1]) + ", " + str(drone.state.location[2]) + ", " +
                              str(drone.state.velocity[0]) + ", " + str(drone.state.velocity[1]) + ", " + str(drone.state.velocity[2]))
                        # Change drone heading towards target detection
                        drone.state.velocity = [drone.cruise_speed * math.cos(targetHeading), drone.cruise_speed * math.sin(targetHeading), 0]

print("Simulation complete.")



NameError: name 'sys' is not defined